### Algoritmo

# Trabajo Practico 2 - Grupo 04 - Backbenchers

**Integrantes:**


* Catalina Basso
* Cristobal Alvarez
* Lucas Ruiz
* Dalmiro Vilaplana  

## Lectura CSV


In [ ]:
#colocamos aqui todos los imports
from google.colab import drive
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
!pip install nltk
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import ComplementNB
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
from sklearn.naive_bayes import BernoulliNB
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import re
import string
nltk.download('stopwords')
import joblib
from nltk.stem import SnowballStemmer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Eliminar puntuación y convertir a minúsculas
stop_words = set(stopwords.words('spanish'))
def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub("\d+", "", text)
    text = text.strip()
    return text

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

def limpiar_texto_leman(texto):
    texto = re.sub(r'[^\w\s]', '', texto)
    texto = texto.lower()
    texto = re.sub(r'\d+', '', texto)

    palabras = texto.split()
    palabras = [palabra for palabra in palabras if palabra not in stop_words]
    lemmatizer = WordNetLemmatizer()
    palabras = [lemmatizer.lemmatize(palabra) for palabra in palabras]
    return ' '.join(palabras)

Primero cargamos y vemos nuestro dataset

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tp2_train.csv')
train.head()

ID                                          review_es sentimiento
0   0  Uno de los otros críticos ha mencionado que de...    positivo
1   1  Una pequeña pequeña producción.La técnica de f...    positivo
2   2  Pensé que esta era una manera maravillosa de p...    positivo
3   3  Básicamente, hay una familia donde un niño peq...    negativo
4   4  El "amor en el tiempo" de Petter Mattei es una...    positivo

In [ ]:
train.shape

(50000, 3)

In [ ]:
test_kaggle = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tp2_test.csv')
test_kaggle.head()

ID                                          review_es
0  60000  La mayor virtud de esta película es su existen...
1  60001  No soy un experto cinéfilo, pero pocas veces m...
2  60002  Si no eres un incondicional del humor estilo T...
3  60003  No sé qué está pasando, si la gente se deja ll...
4  60004  Pero cuando amanece,y me quedo solo,siento en ...

In [ ]:
test_kaggle.shape

(8599, 2)

# **Modelo de red neuronal aplicando Keras y Tensor Flow**

## Modelo 1 (Basico)

### Algoritmo

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train['review_es'] = train['review_es'].apply(limpiar_texto_leman)

# Convertir las etiquetas de 'sentimiento' a valores numéricos
label_encoder = LabelEncoder()
train['sentimiento'] = label_encoder.fit_transform(train['sentimiento'])

# Tokenizar y convertir las reseñas de texto en secuencias numéricas
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train['review_es'])
sequences = tokenizer.texts_to_sequences(train['review_es'])

# Rellenar las secuencias para que todas tengan la misma longitud
padded_sequences = pad_sequences(sequences, maxlen=200, padding='post', truncating='post')

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, train['sentimiento'], test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)



(40000, 200) (10000, 200) (40000,) (10000,)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Definir el modelo
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=200),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(32),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/10
625/625 [==============================] - 205s 319ms/step - loss: 0.6946 - accuracy: 0.5061 - val_loss: 0.6946 - val_accuracy: 0.4960
Epoch 2/10
625/625 [==============================] - 196s 314ms/step - loss: 0.6936 - accuracy: 0.4974 - val_loss: 0.6941 - val_accuracy: 0.4961
Epoch 3/10
625/625 [==============================] - 195s 312ms/step - loss: 0.6932 - accuracy: 0.5021 - val_loss: 0.6932 - val_accuracy: 0.4961
Epoch 4/10
625/625 [==============================] - 204s 326ms/step - loss: 0.6923 - accuracy: 0.5037 - val_loss: 0.6922 - val_accuracy: 0.5127
Epoch 5/10
625/625 [==============================] - 209s 334ms/step - loss: 0.6880 - accuracy: 0.5252 - val_loss: 0.6893 - val_accuracy: 0.5133
Epoch 6/10
625/625 [==============================] - 207s 331ms/step - loss: 0.6618 - accuracy: 0.5905 - val_loss: 0.5701 - val_accuracy: 0.7412
Epoch 7/10
625/625 [==============================] - 205s 329ms/step - loss: 0.5942 - accuracy: 0.6989 - val_loss: 0.5132 -

In [ ]:
predictions = model.predict(X_test)
predictions = np.round(predictions).flatten()

report = classification_report(y_test, predictions)
print(report)

313/313 [==============================] - 21s 64ms/step
              precision    recall  f1-score   support

           0       0.81      0.89      0.85      4961
           1       0.88      0.80      0.84      5039

    accuracy                           0.84     10000
   macro avg       0.85      0.84      0.84     10000
weighted avg       0.85      0.84      0.84     10000



In [ ]:
model.save('keras_tensor_flow_basico.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Prediccion para el modelo de test dado

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

test_kaggle['review_es'] = test_kaggle['review_es'].apply(limpiar_texto_leman)

test_sequences = tokenizer.texts_to_sequences(test_kaggle['review_es'])
test_padded_sequences = pad_sequences(test_sequences, maxlen=200, padding='post', truncating='post')

model = load_model('keras_tensor_flow_basico.h5')

predictions = model.predict(test_padded_sequences)
predicted_labels = (predictions > 0.5).astype('int32')
predicted_labels_text = label_encoder.inverse_transform(predicted_labels.flatten())
test_kaggle['sentimiento'] = predicted_labels_text

columnas_mostrar = ["ID", "sentimiento"]
red_keras_tensor_flow_basico = test_kaggle[columnas_mostrar]
red_keras_tensor_flow_basico.to_csv('red_neuronal_keras_tensor_flow_basico.csv', index=False)
print(red_keras_tensor_flow_basico.head())

269/269 [==============================] - 17s 60ms/step
      ID sentimiento
0  60000    negativo
1  60001    negativo
2  60002    negativo
3  60003    negativo
4  60004    negativo


## Modelo 2 (Basico) sin limpieza de texto

### Algoritmo

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convertir las etiquetas de 'sentimiento' a valores numéricos
label_encoder = LabelEncoder()
train['sentimiento'] = label_encoder.fit_transform(train['sentimiento'])

# Tokenizar y convertir las reseñas de texto en secuencias numéricas
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train['review_es'])
sequences = tokenizer.texts_to_sequences(train['review_es'])

# Rellenar las secuencias para que todas tengan la misma longitud
padded_sequences = pad_sequences(sequences, maxlen=200, padding='post', truncating='post')

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, train['sentimiento'], test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(40000, 200) (10000, 200) (40000,) (10000,)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Definir el modelo
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=200),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(32),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/10
625/625 [==============================] - 213s 325ms/step - loss: 0.6932 - accuracy: 0.5034 - val_loss: 0.6897 - val_accuracy: 0.5137
Epoch 2/10
625/625 [==============================] - 204s 327ms/step - loss: 0.6964 - accuracy: 0.5069 - val_loss: 0.6813 - val_accuracy: 0.5785
Epoch 3/10
625/625 [==============================] - 198s 317ms/step - loss: 0.6800 - accuracy: 0.5658 - val_loss: 0.6800 - val_accuracy: 0.5631
Epoch 4/10
625/625 [==============================] - 206s 330ms/step - loss: 0.6853 - accuracy: 0.5282 - val_loss: 0.6875 - val_accuracy: 0.5241
Epoch 5/10
625/625 [==============================] - 206s 329ms/step - loss: 0.6717 - accuracy: 0.5471 - val_loss: 0.7583 - val_accuracy: 0.5192
Epoch 6/10
625/625 [==============================] - 202s 324ms/step - loss: 0.6272 - accuracy: 0.6130 - val_loss: 0.5008 - val_accuracy: 0.7727
Epoch 7/10
625/625 [==============================] - 200s 320ms/step - loss: 0.4962 - accuracy: 0.7872 - val_loss: 0.4843 -

In [ ]:
predictions = model.predict(X_test)
predictions = np.round(predictions).flatten()

report = classification_report(y_test, predictions)
print(report)

313/313 [==============================] - 20s 62ms/step
              precision    recall  f1-score   support

           0       0.78      0.90      0.84      4961
           1       0.88      0.76      0.81      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



In [ ]:
model.save('keras_tensor_flow_basico_2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Prediccion para el modelo de test dado

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

test_sequences = tokenizer.texts_to_sequences(test_kaggle['review_es'])
test_padded_sequences = pad_sequences(test_sequences, maxlen=200, padding='post', truncating='post')

model = load_model('keras_tensor_flow_basico_2.h5')

predictions = model.predict(test_padded_sequences)
predicted_labels = (predictions > 0.5).astype('int32')
predicted_labels_text = label_encoder.inverse_transform(predicted_labels.flatten())
test_kaggle['sentimiento'] = predicted_labels_text

columnas_mostrar = ["ID", "sentimiento"]
red_keras_tensor_flow_basico_2 = test_kaggle[columnas_mostrar]
red_keras_tensor_flow_basico_2.to_csv('red_neuronal_keras_tensor_flow_basico_2.csv', index=False)
print(red_keras_tensor_flow_basico_2.head())

269/269 [==============================] - 18s 62ms/step
      ID  sentimiento
0  60000            0
1  60001            0
2  60002            0
3  60003            0
4  60004            0


## Modelo 3: Mejorando hiperparametros

### Algoritmo

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2

Esta parte es igual al modelo anterior, lo unico que cambia es el valor de la variable 'maxlen'. Se reduce la longitud de las secuencias, es decir, la cantidad de padding y asi se acelera el entrenamiento.

In [ ]:
train['review_es'] = train['review_es'].apply(limpiar_texto_leman)

# Convertir las etiquetas de 'sentimiento' a valores numéricos
label_encoder = LabelEncoder()
train['sentimiento'] = label_encoder.fit_transform(train['sentimiento'])

# Tokenizar y convertir las reseñas de texto en secuencias numéricas
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train['review_es'])
sequences = tokenizer.texts_to_sequences(train['review_es'])

# Rellenar las secuencias para que todas tengan la misma longitud
padded_sequences = pad_sequences(sequences, maxlen=150, padding='post', truncating='post')  # Reduce maxlen a 150

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, train['sentimiento'], test_size=0.2, random_state=42)

En esta segunda parte aplicamos:
* **Regularización L2:** Ayuda a prevenir el sobreajuste sin aumentar el
tamaño del modelo.
* **Ajuste de Tasa de Aprendizaje:** Una tasa de aprendizaje más baja puede ayudar a mejorar la convergencia del modelo.
* **Tamaño del Lote Ajustado:** Un tamaño de lote más pequeño puede mejorar la eficiencia del entrenamiento.

In [ ]:
# Definir el modelo
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=150),  # Reduce maxlen a 150
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(32, kernel_regularizer=l2(0.01)),  # Añadir regularización L2
    Dropout(0.5),
    Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))  # Añadir regularización L2
])

# Compilar el modelo con una tasa de aprendizaje ajustada
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Entrenar el modelo con un tamaño de lote ajustado
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)  # Reduce batch_size a 32

# Guardar el modelo entrenado
model.save('keras_tensor_flow_mejorado_1.h5')

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/10
1250/1250 [==============================] - 270s 210ms/step - loss: 0.7311 - accuracy: 0.4978 - val_loss: 0.6932 - val_accuracy: 0.4961
Epoch 2/10
1250/1250 [==============================] - 256s 205ms/step - loss: 0.6932 - accuracy: 0.5027 - val_loss: 0.6933 - val_accuracy: 0.4961
Epoch 3/10
1250/1250 [==============================] - 276s 221ms/step - loss: 0.6932 - accuracy: 0.4999 - val_loss: 0.6932 - val_accuracy: 0.4961
Epoch 4/10
1250/1250 [==============================] - 265s 212ms/step - loss: 0.6932 - accuracy: 0.4997 - val_loss: 0.6932 - val_accuracy: 0.4961
Epoch 5/10
1250/1250 [==============================] - 262s 210ms/step - loss: 0.6932 - accuracy: 0.5015 - val_loss: 0.6931 - val_accuracy: 0.5039
Epoch 6/10
1250/1250 [==============================] - 263s 211ms/step - loss: 0.6932 - accuracy: 0.5178 - val_loss: 0.6900 - val_accuracy: 0.4962
Epoch 7/10
1250/1250 [==============================] - 267s 213ms/step - loss: 0.7271 - accuracy: 0.5141 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - 20s 62ms/step - loss: 0.6770 - accuracy: 0.5593
Loss: 0.6770331263542175, Accuracy: 0.5593000054359436


In [ ]:
predictions = model.predict(X_test)
predictions = np.round(predictions).flatten()

report = classification_report(y_test, predictions)
print(report)

313/313 [==============================] - 19s 59ms/step
              precision    recall  f1-score   support

           0       0.53      0.98      0.69      4961
           1       0.89      0.14      0.25      5039

    accuracy                           0.56     10000
   macro avg       0.71      0.56      0.47     10000
weighted avg       0.71      0.56      0.47     10000



### Prediccion para el modelo de test dado

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

test_kaggle['review_es'] = test_kaggle['review_es'].apply(limpiar_texto_leman)

test_sequences = tokenizer.texts_to_sequences(test_kaggle['review_es'])
test_padded_sequences = pad_sequences(test_sequences, maxlen=150, padding='post', truncating='post')

model = load_model('keras_tensor_flow_mejorado_1.h5')

predictions = model.predict(test_padded_sequences)
predicted_labels = (predictions > 0.5).astype('int32')
predicted_labels_text = label_encoder.inverse_transform(predicted_labels.flatten())
test_kaggle['sentimiento'] = predicted_labels_text

columnas_mostrar = ["ID", "sentimiento"]
red_keras_tensor_flow_mejorado_1 = test_kaggle[columnas_mostrar]
red_keras_tensor_flow_mejorado_1.to_csv('red_neuronal_keras_tensor_flow_mejorado_1.csv', index=False)
print(red_keras_tensor_flow_mejorado_1.head())

269/269 [==============================] - 15s 52ms/step
      ID  sentimiento
0  60000            0
1  60001            0
2  60002            0
3  60003            0
4  60004            0


## Modelo 4: GloVe

### Algoritmo

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import pickle

train['review_es'] = train['review_es'].apply(limpiar_texto_leman)

# Convertir las etiquetas de 'sentimiento' a valores numéricos
label_encoder = LabelEncoder()
train['sentimiento'] = label_encoder.fit_transform(train['sentimiento'])

# Tokenizar y convertir las reseñas de texto en secuencias numéricas
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train['review_es'])
sequences = tokenizer.texts_to_sequences(train['review_es'])

# Rellenar las secuencias para que todas tengan la misma longitud
padded_sequences = pad_sequences(sequences, maxlen=150, padding='post', truncating='post')

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, train['sentimiento'], test_size=0.2, random_state=42)

In [ ]:
# Cargar embeddings de GloVe
embedding_index = {}
with open('/content/drive/MyDrive/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Crear una matriz de embeddings
embedding_dim = 100
word_index = tokenizer.word_index
num_words = min(10000, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i >= 10000:
        continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/glove.6B.100d.txt'

In [ ]:
# Definir el modelo
model = Sequential([
    Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix], input_length=150, trainable=False),
    LSTM(64, return_sequences=True),
    BatchNormalization(),  # Normalización por lotes
    Dropout(0.5),
    LSTM(32, kernel_regularizer=l2(0.01)),  # Regularización L2
    BatchNormalization(),  # Normalización por lotes
    Dropout(0.5),
    Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))  # Regularización L2
])

# Compilar el modelo con una tasa de aprendizaje ajustada
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Entrenar el modelo con un tamaño de lote ajustado
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# Guardar el modelo entrenado
model.save('keras_tensor_flow_glove.h5')

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

In [ ]:
predictions = model.predict(X_test)
predictions = np.round(predictions).flatten()

report = classification_report(y_test, predictions)
print(report)

### Prediccion para el modelo de test dado

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

test_kaggle['review_es'] = test_kaggle['review_es'].apply(limpiar_texto_leman)

test_sequences = tokenizer.texts_to_sequences(test_kaggle['review_es'])
test_padded_sequences = pad_sequences(test_sequences, maxlen=150, padding='post', truncating='post')

model = load_model('keras_tensor_flow_glove.h5')

predictions = model.predict(test_padded_sequences)
predicted_labels = (predictions > 0.5).astype('int32')
predicted_labels_text = label_encoder.inverse_transform(predicted_labels.flatten())

test_kaggle['sentimiento'] = predicted_labels_text

def convert_sentiment(value):
    return 'positivo' if value == 1 else 'negativo'

test_kaggle['sentimiento'] = test_kaggle['sentimiento'].apply(convert_sentiment)

columnas_mostrar = ["ID", "sentimiento"]
keras_tensor_flow_glove = test_kaggle[columnas_mostrar]
keras_tensor_flow_glove.to_csv('keras_tensor_flow_glove.csv', index=False)
print(keras_tensor_flow_glove.head())

## Modelo 5: Bag of words


### Algoritmo

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import gc

In [ ]:
label_encoder = LabelEncoder()
train['sentimiento'] = label_encoder.fit_transform(train['sentimiento'])

In [ ]:
# Crear un vectorizador de palabras (Bag of Words)
vectorizer = CountVectorizer(max_features=5000)  # Reduce max_features para ahorrar memoria
X = vectorizer.fit_transform(train['review_es']).toarray()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, train['sentimiento'], test_size=0.2, random_state=42)

In [ ]:
# Liberar memoria no utilizada
del train, X
gc.collect()

55132

In [ ]:
# Definir el modelo
model = Sequential([
    Dense(256, activation='relu', input_shape=(5000,)),  # Reducir el número de neuronas
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Entrenar el modelo con un batch_size más pequeño
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)  # Reducir batch_size

Epoch 1/10
1250/1250 [==============================] - 43s 33ms/step - loss: 0.4068 - accuracy: 0.8237 - val_loss: 0.3309 - val_accuracy: 0.8551
Epoch 2/10
1250/1250 [==============================] - 38s 30ms/step - loss: 0.2859 - accuracy: 0.8810 - val_loss: 0.3231 - val_accuracy: 0.8632
Epoch 3/10
1250/1250 [==============================] - 37s 30ms/step - loss: 0.2100 - accuracy: 0.9176 - val_loss: 0.3518 - val_accuracy: 0.8579
Epoch 4/10
1250/1250 [==============================] - 40s 32ms/step - loss: 0.1389 - accuracy: 0.9491 - val_loss: 0.4189 - val_accuracy: 0.8581
Epoch 5/10
1250/1250 [==============================] - 34s 27ms/step - loss: 0.0932 - accuracy: 0.9666 - val_loss: 0.5115 - val_accuracy: 0.8626
Epoch 6/10
1250/1250 [==============================] - 37s 30ms/step - loss: 0.0714 - accuracy: 0.9748 - val_loss: 0.5722 - val_accuracy: 0.8515
Epoch 7/10
1250/1250 [==============================] - 37s 30ms/step - loss: 0.0626 - accuracy: 0.9777 - val_loss: 0.5608 -

In [ ]:
# Guardar el modelo entrenado
model.save('keras_tensor_flow_bow.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
predictions = model.predict(X_test)
predictions = np.round(predictions).flatten()

report = classification_report(y_test, predictions)
print(report)

313/313 [==============================] - 2s 7ms/step
              precision    recall  f1-score   support

           0       0.85      0.86      0.86      4961
           1       0.86      0.86      0.86      5039

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



### Prediccion para el modelo de test dado

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import load_model

# Vectorizar el conjunto de datos de prueba utilizando el mismo CountVectorizer
vectorizer = CountVectorizer(max_features=5000)
X_test = vectorizer.fit_transform(test_kaggle['review_es']).toarray()

# Cargar el modelo entrenado
model = load_model('keras_tensor_flow_bow.h5')

# Hacer predicciones
predictions = model.predict(X_test)

# Convertir las predicciones a etiquetas (0 y 1)
predicted_labels = (predictions > 0.5).astype('int32')

# Convertir 0 y 1 a 'negativo' y 'positivo'
def convert_sentiment(value):
    return 'positivo' if value == 1 else 'negativo'

predicted_labels_text = [convert_sentiment(label) for label in predicted_labels]

# Agregar las predicciones al DataFrame de test
test_kaggle['sentimiento'] = predicted_labels_text

columnas_mostrar = ["ID", "sentimiento"]
keras_tensor_flow_bow = test_kaggle[columnas_mostrar]
keras_tensor_flow_bow.to_csv('keras_tensor_flow_bow.csv', index=False)

print(keras_tensor_flow_bow.head())

269/269 [==============================] - 2s 7ms/step
      ID sentimiento
0  60000    negativo
1  60001    positivo
2  60002    negativo
3  60003    positivo
4  60004    negativo


## Modelo 6: TF-IDF

### Algoritmo

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import gc

In [ ]:
label_encoder = LabelEncoder()
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tp2_train.csv')
train['sentimiento'] = label_encoder.fit_transform(train['sentimiento'])

In [ ]:
# Crear un vectorizador TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_es']).toarray()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, train['sentimiento'], test_size=0.2, random_state=42)

In [ ]:
# Liberar memoria no utilizada
del train, X
gc.collect()

59515

In [ ]:
# Definir el modelo
model = Sequential([
    Dense(256, activation='relu', input_shape=(5000,)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Entrenar el modelo con un batch_size más pequeño
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/10
1250/1250 [==============================] - 35s 27ms/step - loss: 0.3550 - accuracy: 0.8448 - val_loss: 0.2874 - val_accuracy: 0.8770
Epoch 2/10
1250/1250 [==============================] - 33s 27ms/step - loss: 0.2571 - accuracy: 0.8949 - val_loss: 0.2948 - val_accuracy: 0.8736
Epoch 3/10
1250/1250 [==============================] - 33s 26ms/step - loss: 0.2092 - accuracy: 0.9182 - val_loss: 0.3080 - val_accuracy: 0.8748
Epoch 4/10
1250/1250 [==============================] - 35s 28ms/step - loss: 0.1523 - accuracy: 0.9452 - val_loss: 0.3396 - val_accuracy: 0.8719
Epoch 5/10
1250/1250 [==============================] - 34s 27ms/step - loss: 0.0949 - accuracy: 0.9688 - val_loss: 0.4327 - val_accuracy: 0.8709
Epoch 6/10
1250/1250 [==============================] - 34s 27ms/step - loss: 0.0629 - accuracy: 0.9792 - val_loss: 0.4612 - val_accuracy: 0.8680
Epoch 7/10
1250/1250 [==============================] - 32s 26ms/step - loss: 0.0450 - accuracy: 0.9843 - val_loss: 0.5371 -

In [ ]:
# Guardar el modelo entrenado
model.save('keras_tensor_flow_tf_idf.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
predictions = model.predict(X_test)
predictions = np.round(predictions).flatten()

report = classification_report(y_test, predictions)
print(report)

313/313 [==============================] - 2s 7ms/step
              precision    recall  f1-score   support

           0       0.88      0.84      0.86      4961
           1       0.85      0.89      0.87      5039

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



### Prediccion para el modelo de test dado

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import load_model

# Vectorizar el conjunto de datos de prueba utilizando el mismo CountVectorizer
vectorizer = CountVectorizer(max_features=5000)
X_test = vectorizer.fit_transform(test_kaggle['review_es']).toarray()

# Cargar el modelo entrenado
model = load_model('keras_tensor_flow_tf_idf.h5')

# Hacer predicciones
predictions = model.predict(X_test)

# Convertir las predicciones a etiquetas (0 y 1)
predicted_labels = (predictions > 0.5).astype('int32')

# Convertir 0 y 1 a 'negativo' y 'positivo'
def convert_sentiment(value):
    return 'positivo' if value == 1 else 'negativo'

predicted_labels_text = [convert_sentiment(label) for label in predicted_labels]

# Agregar las predicciones al DataFrame de test
test_kaggle['sentimiento'] = predicted_labels_text

columnas_mostrar = ["ID", "sentimiento"]
keras_tensor_flow_tf_idf = test_kaggle[columnas_mostrar]
keras_tensor_flow_tf_idf.to_csv('keras_tensor_flow_tf_idf.csv', index=False)

print(keras_tensor_flow_tf_idf.head())

269/269 [==============================] - 2s 8ms/step
      ID sentimiento
0  60000    positivo
1  60001    positivo
2  60002    positivo
3  60003    positivo
4  60004    positivo


##Modelo 7: Early Stopping

En este modelo, intentaremos usar ambas estrategias utilizadas previamente: bag of words y TF-IDF junto con el metodo de regularizacion Early Stopping, para evitar overfitting. Además, incrementamos max_features a 10000 y usamos un bigrama para capturar más informacion del texto. Tambien, ajustaremos la tasa de learning rate a 0,0001

###Algoritmo

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
label_encoder = LabelEncoder()
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tp2_train.csv')
train['sentimiento'] = label_encoder.fit_transform(train['sentimiento'])

In [ ]:
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X = vectorizer.fit_transform(train['review_es']).toarray()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, train['sentimiento'], test_size=0.2, random_state=42)

In [ ]:
# Defino el modelo
model = Sequential([
    Dense(512, activation='relu', input_shape=(10000,)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=64, callbacks=[early_stopping])

# Guardar el modelo entrenado
model.save('keras_tensor_flow_tf_idf_v2.h5')

In [ ]:
# Evaluar el modelo en el conjunto de prueba
y_pred = model.predict(X_test)
predictions = np.round(y_pred).flatten()

# Mostrar el reporte de clasificación
report = classification_report(y_test, predictions)
print(report)

### Predicción para el modelo de test dado

In [ ]:
# Preprocesar las nuevas críticas
test_kaggle['review_es'] = test_kaggle['review_es'].apply(limpiar_texto)

# Convertir las nuevas críticas en TF-IDF utilizando el vectorizador entrenado
X_test_kaggle = vectorizer.transform(test_kaggle['review_es']).toarray()

# Cargar el modelo entrenado
model = load_model('keras_tensor_flow_tf_idf_v2.h5')

# Hacer predicciones en los datos de prueba
y_pred_kaggle = model.predict(X_test_kaggle)
predicted_labels = (y_pred_kaggle > 0.5).astype('int32')

# Convertir 0 y 1 a 'negativo' y 'positivo'
def convert_sentiment(value):
    return 'positivo' if value == 1 else 'negativo'

predicted_labels_text = [convert_sentiment(label) for label in predicted_labels]

# Agregar las predicciones al DataFrame de test
test_kaggle['sentimiento'] = predicted_labels_text

# Guardar el DataFrame con las predicciones a un nuevo archivo CSV
columnas_mostrar = ["ID", "sentimiento"]
keras_tensor_flow_tf_idf_v2 = test_kaggle[columnas_mostrar]
keras_tensor_flow_tf_idf_v2.to_csv('/content/drive/MyDrive/Colab Notebooks/keras_tensor_flow_tf_idf_v2.csv', index=False)


## Modelo 8: Embedding y Dense Layers

###Algoritmo

In [ ]:
pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import tensorflow.keras as keras
from keras.preprocessing import text, sequence

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
train['review_es'] = train['review_es'].apply(limpiar_texto_leman)

In [ ]:
# Preprocesamiento y tokenización
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['review_es'])

X = tokenizer.texts_to_sequences(train['review_es'])
X = pad_sequences(X, maxlen=128)

# Codificación de etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train['sentimiento'])

In [ ]:
# Dividir el dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Definir el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=128),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

In [ ]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluación del modelo
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Loss: {loss}, Accuracy: {accuracy}")

In [ ]:
from sklearn.metrics import classification_report, f1_score

# Predicciones en el conjunto de validación
y_val_pred = model.predict(X_val)
y_val_pred = (y_val_pred > 0.5).astype(int)

# Métricas adicionales
print(classification_report(y_val, y_val_pred))
print(f"F1-Score: {f1_score(y_val, y_val_pred)}")

### Prediccion para el modelo de test dado

In [ ]:
test_kaggle['review_es'] = test_kaggle['review_es'].apply(limpiar_texto_leman)

X_test = tokenizer.texts_to_sequences(test_kaggle['review_es'])
X_test = pad_sequences(X_test, maxlen=128)

In [ ]:
y_test_pred = model.predict(X_test)
y_test_pred = (y_test_pred > 0.5).astype(int)

predictions = pd.DataFrame({
    'ID': test_kaggle['ID'],
    'sentimiento': y_test_pred.flatten()
})

def convert_sentiment(value):
    return 'positivo' if value == 1 else 'negativo'

predictions['sentimiento'] = predictions['sentimiento'].apply(convert_sentiment)

predictions.to_csv('keras_flow_embedding_dense.csv', index=False)

##Modelo 9


###Algoritmo

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tp2_train.csv')

In [ ]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords)
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords)


In [ ]:
train['sentimiento'] = train['sentimiento'].map({'positivo': 1, 'negativo': 0})

In [ ]:
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['review_clean'])

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(train['review_clean'])
X_test_seq = tokenizer.texts_to_sequences(test_kaggle['review_clean'])

In [ ]:
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [ ]:
y = train['sentimiento']
y_train_cat = to_categorical(y, num_classes=2)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_pad, y_train_cat, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
optimizer = Adam(learning_rate=0.001)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping])

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print("Accuracy en el conjunto de validación:", accuracy)

###Prediccion para el modelo de test dado

In [ ]:
y_pred_kaggle = model.predict(X_test_pad)
y_pred_kaggle = np.argmax(y_pred_kaggle, axis=1)

In [ ]:
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})

def convert_sentiment(value):
    return 'positivo' if value == 1 else 'negativo'

submission['sentimiento'] = submission['sentimiento'].apply(convert_sentiment)

In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/submission_redes_neu_3.csv', index=False)

##Modelo 10

###Algoritmo

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tp2_train.csv')

In [ ]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords)
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords)

In [ ]:
train['sentimiento'] = train['sentimiento'].map({'positivo': 1, 'negativo': 0})

In [ ]:
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['review_clean'])

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(train['review_clean'])
X_test_seq = tokenizer.texts_to_sequences(test_kaggle['review_clean'])

In [ ]:
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [ ]:
y = train['sentimiento']
y_train_cate = to_categorical(y, num_classes=2)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_pad, y_train_cat, test_size=0.2, random_state=42)



In [ ]:
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
optimizer = Adam(learning_rate=0.001)  # Experimenta con diferentes learning rates
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])



In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping])



In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print("Accuracy en el conjunto de validación:", accuracy)

In [ ]:
y_pred_kaggle = model.predict(X_test_pad)
y_pred_kaggle = np.argmax(y_pred_kaggle, axis=1)

###Prediccion para el modelo de test dado

In [ ]:
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})

def convert_sentiment(value):
    return 'positivo' if value == 1 else 'negativo'

submission['sentimiento'] = submission['sentimiento'].apply(convert_sentiment)

In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/submission_redes_neu_1.csv', index=False)
